- Update: 2023.04.12

In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

# = = = = = = = = = = = = = = = = = =

chromedriver = '../../chromedriver.exe'

# = = = = = = = = = = = = = = = = = =

chrome_options = ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # Chrome正受到自动测试软件的控制
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # window.navigator.webdriver=false
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
chrome_options.add_argument('--proxy-server=http://j933.kdltps.com:15818')
chrome_options.add_argument('blink-settings=imagesEnabled=false')
chrome_options.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu-ngk_laser_iridium_spark_plugs.xlsx')
list_product = df_menu['Product'].to_list()
print('总数量：' + str(len(list_product)))
print()

# = = = = = = = = = = = = = = = = = =

df = pd.DataFrame(columns=['Item',
                           '标题',
                           '图片',
                           'Brand',
                           'Part Number',
                           'UPC',
                           '车型',
                           'Electrode Tip Material',
                           'Electrode Core Material',
                           '链接',
                           '图片链接'])

# = = = = = = = = = = = = = = = = = =

work = Queue()
for product in list_product:
    work.put_nowait(product)

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        product = work.get_nowait()
        url = product
        driver = Chrome(service=Service(chromedriver), options=chrome_options)
        driver.maximize_window()
        time.sleep(0.3)
        driver.maximize_window()
        
        for i in range(11):
            try:
                driver.get(url)
                driver.maximize_window()

                # = = = = = = = = = = = = = = = = = =

                WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="part-detail-title"]')))
                title = driver.find_element(By.XPATH, '//*[@class="part-detail-title"]').text
                item = driver.find_element(By.XPATH, '//*[@class="part-detail-title-number"]').text.replace('Part Number: ', '')
                pic = driver.find_element(By.XPATH, '//*[@id="primaryImage"]').get_attribute('src')
                
                # = = = = = = = = = = = = = = = = = =

                brand = ''
                part_number = ''
                upc = ''
                electrode_tip = ''
                electrode_core = ''
                WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="small-12 medium-9 large-6"]')))
                overview_left = driver.find_elements(By.XPATH, '//*[@class="small-12 medium-9 large-6"]')
                overview_right = driver.find_elements(By.XPATH, '//*[@class="small-12 medium-15 large-18"]')
                for j, k in zip(overview_left, overview_right):
                    if j.text == 'Brand:':
                        brand = k.text
                    elif j.text == 'Manufacturer\'s Part Number:':
                        part_number = k.text
                    elif j.text == 'UPC:':
                        upc = k.text
                    elif j.text == 'Electrode Tip Material:':
                        electrode_tip = k.text
                    elif j.text == 'Electrode Core Material:':
                        electrode_core = k.text
                
                # = = = = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Item': [str(item)],
                                        '标题': [str(title)],
                                        '图片': [''],
                                        'Brand': [str(brand)],
                                        'Part Number': [str(part_number)],
                                        'UPC': [str(upc)],
                                        '车型': [''],
                                        'Electrode Tip Material': [str(electrode_tip)],
                                        'Electrode Core Material': [str(electrode_core)],
                                        '链接': [str(url)],
                                        '图片链接': [str(pic)]})
                
                break

            except:
                df_temp = pd.DataFrame({'Item': [''],
                                        '标题': [''],
                                        '图片': [''],
                                        'Brand': [''],
                                        'Part Number': [''],
                                        'UPC': [''],
                                        '车型': [''],
                                        'Electrode Tip Material': [''],
                                        'Electrode Core Material': [''],
                                        '链接': [str(url)],
                                        '图片链接': ['']})
                continue

        # = = = = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
        print(product, ' [' + str(i) + '] - 剩余数量：' + str(work.qsize()))
        driver.close()
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(4):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df.to_excel('./crawler.xlsx', index=False)
print()
print('搞定')

batch：2 - 总数量：7

https://www.summitracing.com/parts/ngk-91234  [0] - 剩余数量：3
https://www.summitracing.com/parts/ngk-4912  [0] - 剩余数量：2
https://www.summitracing.com/parts/ngk-96412  [0] - 剩余数量：1
https://www.summitracing.com/parts/ngk-4214  [0] - 剩余数量：0
https://www.summitracing.com/parts/ngk-5899  [0] - 剩余数量：0
https://www.summitracing.com/parts/ngk-90856  [0] - 剩余数量：0
https://www.summitracing.com/parts/ngk-93623  [0] - 剩余数量：0

搞定


D:\Lennon\python\venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
